# Определение выгодного тарифа для телеком компании

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откроем и изучим файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

In [44]:
df = pd.read_csv("/datasets/users_behavior.csv")

display(df.shape)
display(df.head(30)) 

(3214, 5)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Столбцы содержат следующую информацию: 
    
сalls — количество звонков,

minutes — суммарная длительность звонков в минутах,

messages — количество sms-сообщений,

mb_used — израсходованный интернет-трафик в Мб,

is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Посмотрим на типы столбцов

In [3]:
df.info()
print('')
print('Количество полных дубликатов:', df.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 3214 entries, 0 to 3213

Data columns (total 5 columns):

 #   Column    Non-Null Count  Dtype  

---  ------    --------------  -----  

 0   calls     3214 non-null   float64

 1   minutes   3214 non-null   float64

 2   messages  3214 non-null   float64

 3   mb_used   3214 non-null   float64

 4   is_ultra  3214 non-null   int64  

dtypes: float64(4), int64(1)

memory usage: 125.7 KB



Количество полных дубликатов: 0


Посчитаем количество пропусков в наборе данных

In [4]:
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [5]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


## Разобьем данные на выборки

Разделим датасет на обучающую и тестовую выборки

In [6]:
train, test = train_test_split(df, test_size=0.25, random_state=12345)

In [7]:
features_one = test.drop('is_ultra', axis=1)
target_one = test['is_ultra']

Создадим переменные для признаков и целевого признака

In [8]:
features_valid, features_test, target_valid, target_test = train_test_split(features_one, target_one, test_size=0.5, random_state=12345)

In [9]:
features_train = train.drop('is_ultra', axis=1)
target_train = train['is_ultra']

In [10]:
print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)

(2410, 4)

(2410,)

(402, 4)

(402,)


## Исследуем модели

Перебем гиперпараметры в дереве решений через цикл Tree Classifier 

In [11]:
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth = depth)

    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid) 

    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))

max_depth = 1 : 0.763681592039801

max_depth = 2 : 0.7935323383084577

max_depth = 3 : 0.7985074626865671

max_depth = 4 : 0.7985074626865671

max_depth = 5 : 0.7985074626865671


Подходящая max_depth = 3

Качество = 0.7985074626865671

Проверим качество с помощью логистической регрессии 

In [12]:
model = LogisticRegression()
model.fit(features_train, target_train)
accuracy = model.score(features_valid, target_valid)
print("Качество:", accuracy)

Качество: 0.7039800995024875


Проверим качество с помощью Random Forest 

In [13]:
best_accuracy = 0
best_depth = 0
for depth in range(1,10):
    for est in range(5,50,5):
        for sample in range(2,5):
            model = RandomForestClassifier(max_depth=depth, n_estimators=est, min_samples_leaf=sample, random_state=412)
            model.fit(features_train, target_train)
            accuracy = model.score(features_valid, target_valid)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_depth = depth
                best_est = est
                best_sample = sample
print("Глубина дерева:", best_depth, "    Количество деревьев:", best_est, "   минимальное количество объектов в узле:", best_sample, "  Качество:", best_accuracy)

Глубина дерева: 6     Количество деревьев: 20    минимальное количество объектов в узле: 3   Качество: 0.8109452736318408


Проверены три модели: деревья решений, логистическая регрессия и случайный лес.

Самое высокое качество модели (0.810) достигается при использовании Random Forest с гиперпараметрами max_depth=7 , n_estimators=20 и min_samples_leaf=3

## Проверим модель на тестовой выборке

Проверим качество с помощью Tree Classifier

In [14]:
model = DecisionTreeClassifier(max_depth=8, random_state=412)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.7960199004975125


Проверим качество с помощью логистической регрессии

In [15]:
model = LogisticRegression()
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.7039800995024875


Проверим качество с помощью Random Forest

In [16]:
model = RandomForestClassifier(max_depth=7, n_estimators=40, min_samples_leaf=3, random_state=228)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.8084577114427861


Самое высокое качество модели (0.808) достигается при использовании Random Forest с гиперпараметрами max_depth=7 , n_estimators=40 и min_samples_leaf=3

## Вывод

Проверены три модели: деревья решений, логистическая регрессия и случайный лес.


Самое высокое качество модели (0.808) достигается при использовании Random Forest с гиперпараметрами max_depth=7 , n_estimators=40 и min_samples_leaf=3